## Import OGER output

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
%%time

import pandas as pd
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'

CPU times: user 49 µs, sys: 0 ns, total: 49 µs
Wall time: 54.1 µs


In [4]:
cols = ['BiosampleId', 'Biolink', 'BeginTerm', 'EndTerm', 'TokenizedTerm', 'PreferredTerm', 'CURIE', 'NaN1', 'S-Type', 'NaN2', 'UMLS_CUI']
ogerOutput = pd.read_csv('../output/FullNLPOutput.tsv', sep='\t', names=cols )
ogerOutput = ogerOutput.drop(['NaN1', 'NaN2', 'UMLS_CUI'], axis = 1)
print('Total number of rows: ', str(len(ogerOutput)))
ogerOutput.head()
#pd.set_option("display.max_rows", None)
#print(ogerOutput.CURIE.value_counts())

Total number of rows:  2386121


,BiosampleId,Biolink,BeginTerm,EndTerm,TokenizedTerm,PreferredTerm,CURIE,S-Type
0,SAMN10586215,biolink:OntologyClass,16,24,contains,contains,RO:0001019,S1
1,SAMN10586215,biolink:OrganismalEntity,119,127,Bacteria,Bacteria,NCBITaxon:2,S1
2,SAMN05720810,biolink:OntologyClass,50,57,part of,part of,BFO:0000050,S1
3,SAMN05720810,biolink:OntologyClass,79,90,Agriculture,agricultural process,ENVO:01001246,S1
4,SAMN05720810,biolink:OntologyClass,79,90,Agriculture,animal husbandry,ENVO:01001248,S1


In [5]:
ogerInput = pd.read_csv('../output/biosampleDescriptionDF.tsv', sep='\t')
ogerInput.BiosampleId = ogerInput.BiosampleId.map(lambda x: x.replace('BIOSAMPLE:',''))
ogerInput = ogerInput.drop(['StudyId', 'Name', 'Title'], axis = 1)
print('Number of rows: '+ str(len(ogerInput)))
print('Number of unique Ids: '+ str(len(ogerInput.BiosampleId.unique())))
print('Number of unique Desc: '+ str(len(ogerInput.Description.unique())))
ogerInput = ogerInput.drop_duplicates()
print('******After duplicate row removal*****')
print('Number of rows: '+ str(len(ogerInput)))
print('Number of unique Ids: '+ str(len(ogerInput.BiosampleId.unique())))
print('Number of unique Description: '+ str(len(ogerInput.Description.unique())))
ogerInput.head()

Number of rows: 948401
Number of unique Ids: 938333
Number of unique Desc: 59789
******After duplicate row removal*****
Number of rows: 947374
Number of unique Ids: 938333
Number of unique Description: 59789


,Description,BiosampleId
0,Treponema denticola ATCC 35405. Treponema dent...,SAMN02603967
1,Treponema pallidum subsp. pallidum. This organ...,SAMN02604348
2,Campylobacter fetus subsp. venerealis str. Azu...,SAMN02471365
3,Campylobacter jejuni strain NCTC11168. This st...,SAMEA1705929
4,Francisella tularensis subsp. tularensis strai...,SAMEA3138185


In [6]:
print('Unique BiosampleIds: '+ str(len(ogerOutput.BiosampleId.unique())))
print('Unique CURIEs: '+ str(len(ogerOutput.CURIE.unique())))
print('Unique Biolinks: '+ str(len(ogerOutput.Biolink.unique())))
print('Unique Standard Terms: '+ str(len(ogerOutput.PreferredTerm.unique())))

Unique BiosampleIds: 649677
Unique CURIEs: 1686
Unique Biolinks: 4
Unique Standard Terms: 1677


In [ ]:
curieOfInterest = ['ENVO', 'PATO', 'RO']
envoPato = ogerOutput[ogerOutput.CURIE.str.contains('|'.join(curieOfInterest))]
print('Number of rows: '+ str(len(envoPato)))
envoPato.head()

In [ ]:
salineTerms = ['sali', 'salt']
envPatSal = envoPato[envoPato.TokenizedTerm	.str.contains('|'.join(salineTerms))]
print('Number of rows: '+ str(len(envPatSal)))
envPatSal.head()

In [8]:
# Get list of IDs from non-human-sample file
#nonHumanDF = pd.read_csv('../input/non-human-samples.tsv.gz', sep='\t', low_memory=False)
structuredDataDF = pd.read_csv('../input/water-related-samples.tsv.gz', sep='\t', low_memory=False)
print('Number of rows: ', str(len(structuredDataDF)))
structuredDataDF.head()

Number of rows:  18615


,tot_diss_nitro,salinity_meth,tidal_stage,num_replicons,nitrate,rel_to_oxygen,potassium,sodium,env_package,biomass,...,depth,silicate,tot_part_carb,trophic_level,calcium,id,taxonomy_id,taxonomy_name,package,norm_env_package
0,NaN,NaN,NaN,NaN,NaN,NaN,2.33,NaN,water,NaN,...,NaN,NaN,NaN,NaN,87.66,BIOSAMPLE:SAMEA1706329,449393,freshwater metagenome,Generic.1.0,water
1,NaN,NaN,NaN,NaN,NaN,NaN,1.33,NaN,water,NaN,...,NaN,NaN,NaN,NaN,62.28,BIOSAMPLE:SAMEA1706311,449393,freshwater metagenome,Generic.1.0,water
2,NaN,NaN,NaN,NaN,NaN,NaN,2.77,NaN,water,NaN,...,NaN,NaN,NaN,NaN,87.05,BIOSAMPLE:SAMEA1706320,449393,freshwater metagenome,Generic.1.0,water
3,NaN,NaN,NaN,NaN,NaN,NaN,2.52,NaN,water,NaN,...,NaN,NaN,NaN,NaN,126.11,BIOSAMPLE:SAMEA1706300,449393,freshwater metagenome,Generic.1.0,water
4,NaN,NaN,NaN,NaN,NaN,NaN,3.4,NaN,water,NaN,...,NaN,NaN,NaN,NaN,332.17,BIOSAMPLE:SAMEA1706316,449393,freshwater metagenome,Generic.1.0,water


### Get descriptions of non-human-samples only

#### From OGER Input

In [17]:
structDataID = structuredDataDF.id.str.replace('BIOSAMPLE:','')
structDataIdDF = pd.DataFrame(structDataID)
structDataOgerInput = ogerInput.merge(structDataIdDF, how='inner', left_on='BiosampleId', right_on='id')
structDataOgerInput = structDataOgerInput.drop(['id'], axis = 1)
print('Number of rows: '+ str(len(nhOgerInput)))
structDataOgerInput.head()

Number of rows: 243


,Description,BiosampleId
0,Purple microbial (Photo- and chemosynthetic) m...,SAMN02228699
1,Purple microbial (Photo- and chemosynthetic) m...,SAMN02228700
2,Purple microbial (Photo- and chemosynthetic) m...,SAMN02228701
3,Purple microbial (Photo- and chemosynthetic) m...,SAMN02228702
4,Purple microbial (Photo- and chemosynthetic) m...,SAMN02228703


#### From OGER output

In [18]:
structDataOgerOutput = ogerOutput.merge(structDataIdDF, how='inner', left_on='BiosampleId', right_on='id')
structDataOgerOutput = structDataOgerOutput.drop(['id'], axis = 1)
print('Number of row: '+ str(len(structDataOgerOutput)))
structDataOgerOutput.head()

Number of row: 2429


,BiosampleId,Biolink,BeginTerm,EndTerm,TokenizedTerm,PreferredTerm,CURIE,S-Type
0,SAMN02393652,biolink:ChemicalSubstance,19,24,probe,probe,CHEBI:50406,S1
1,SAMN02393652,biolink:OntologyClass,61,65,soil,soil,ENVO:00001998,S1
2,SAMN02393652,biolink:AnatomicalEntity,70,73,gut,digestive tract,UBERON:0001555,S1
3,SAMN02393652,biolink:OntologyClass,70,73,gut,watercourse,ENVO:00000029,S1
4,SAMN02393652,biolink:AnatomicalEntity,70,73,gut,digestive system,UBERON:0001007,S1


### Form Phrases for each biosample

In [11]:
%%time
# Function to phrase-ify
def phrasify(dataFrame, columnNames):
    count = 0
    returnDF = pd.DataFrame(columns=columnNames)
    
    bsIdList = dataFrame.BiosampleId.unique()
    
    for i in bsIdList:
        count += 1
        tmpDF = pd.DataFrame()
        bpList = []
        phrase1 = []
        phrase2 = []
        curie = []

        tmpDF = dataFrame.loc[(dataFrame.BiosampleId == i)]
        
        # Filter: Consider ONLY 3 or more DISTINCT terms AND if an RO CURIE is present
        
        #if len(tmpDF) > 2 \
        #and len(tmpDF.TokenizedTerm.unique()) > 2 \
        #and tmpDF.CURIE.str.contains('RO').any():
            
        tmpDF.sort_values(by=['BeginTerm'], inplace=True)

        for row in tmpDF.iterrows():
            bpList.append(row[1].BeginTerm)
            phrase1.append(row[1].TokenizedTerm)
            phrase2.append(row[1].PreferredTerm)
            curie.append(row[1].CURIE)

        returnDF = returnDF.append(pd.DataFrame([[i, bpList, phrase1, phrase2, curie]], columns=columnNames))
            
        if count % 1000 == 0:
            print('Number of relevant rows: '+ str(len(returnDF))+ ' - '+ datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        
        # *********** FOR DEBUG PURPOSES ONLY ****************
        if count == 5000:
            break;
        
    return returnDF

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11 µs


In [12]:
%%time

pCols = ['BiosampleId', 'BeginPosList', 'Phrase', 'StdPhrase', 'CURIE']
phraseDF = phrasify(nhOgerOutput,pCols)
print('*****Number of rows: '+ str(len(phraseDF)) + ' ********')
phraseDF.head()

*****Number of rows: 238 ********
CPU times: user 1.04 s, sys: 8.34 ms, total: 1.05 s
Wall time: 1.09 s


,BiosampleId,BeginPosList,Phrase,StdPhrase,CURIE
0,SAMN02393652,"[19, 61, 70, 70, 70, 208]","[probe, soil, gut, gut, gut, function]","[probe, soil, digestive tract, watercourse, di...","[CHEBI:50406, ENVO:00001998, UBERON:0001555, E..."
0,SAMN02389811,"[20, 48, 179, 189, 202, 202, 221, 221, 241, 25...","[aquarium, mesocosm, part of, flow, system, sy...","[aquarium, mesocosm, part of, mass wasting flo...","[ENVO:00002196, ENVO:01000620, BFO:0000050, EN..."
0,SAMN02251077,"[65, 98, 188, 198, 216, 231, 259, 259, 273, 273]","[bacteria, lagoon, site, temperate, lagoon, fr...","[Bacteria, lagoon, site, temperate, lagoon, fr...","[NCBITaxon:2, ENVO:00000038, BFO:0000029, ENVO..."
0,SAMEA2620360,"[42, 160, 200, 231, 231]","[seawater, Ocean, all, quality, quality]","[sea water, ocean, root, quality, quality]","[ENVO:00002149, ENVO:00000015, NCBITaxon:1, OB..."
0,SAMN02009298,"[63, 63, 63, 68, 89, 93, 138, 143, 201, 236, 4...","[deep, deep, deep, sea, Sea, rift, halo, strat...","[undersea trough, ocean trench, increased dept...","[ENVO:01000654, ENVO:00000275, OBO:PATO_000159..."


In [13]:
# Add 'BIOSAMPLE:' to the id 
phraseDF.BiosampleId = 'BIOSAMPLE:' + phraseDF.BiosampleId
phraseDF.head()

,BiosampleId,BeginPosList,Phrase,StdPhrase,CURIE
0,BIOSAMPLE:SAMN02393652,"[19, 61, 70, 70, 70, 208]","[probe, soil, gut, gut, gut, function]","[probe, soil, digestive tract, watercourse, di...","[CHEBI:50406, ENVO:00001998, UBERON:0001555, E..."
0,BIOSAMPLE:SAMN02389811,"[20, 48, 179, 189, 202, 202, 221, 221, 241, 25...","[aquarium, mesocosm, part of, flow, system, sy...","[aquarium, mesocosm, part of, mass wasting flo...","[ENVO:00002196, ENVO:01000620, BFO:0000050, EN..."
0,BIOSAMPLE:SAMN02251077,"[65, 98, 188, 198, 216, 231, 259, 259, 273, 273]","[bacteria, lagoon, site, temperate, lagoon, fr...","[Bacteria, lagoon, site, temperate, lagoon, fr...","[NCBITaxon:2, ENVO:00000038, BFO:0000029, ENVO..."
0,BIOSAMPLE:SAMEA2620360,"[42, 160, 200, 231, 231]","[seawater, Ocean, all, quality, quality]","[sea water, ocean, root, quality, quality]","[ENVO:00002149, ENVO:00000015, NCBITaxon:1, OB..."
0,BIOSAMPLE:SAMN02009298,"[63, 63, 63, 68, 89, 93, 138, 143, 201, 236, 4...","[deep, deep, deep, sea, Sea, rift, halo, strat...","[undersea trough, ocean trench, increased dept...","[ENVO:01000654, ENVO:00000275, OBO:PATO_000159..."


In [19]:
#result = ogerInput.merge(phraseDF, how='inner', on='BiosampleId')
result = structuredDataDF.merge(phraseDF, how='left', left_on='id', right_on='BiosampleId')
print('Number of rows: '+ str(len(result)))
result.head()

Number of rows: 18615


,tot_diss_nitro,salinity_meth,tidal_stage,num_replicons,nitrate,rel_to_oxygen,potassium,sodium,env_package,biomass,...,id,taxonomy_id,taxonomy_name,package,norm_env_package,BiosampleId,BeginPosList,Phrase,StdPhrase,CURIE
0,NaN,NaN,NaN,NaN,NaN,NaN,2.33,NaN,water,NaN,...,BIOSAMPLE:SAMEA1706329,449393,freshwater metagenome,Generic.1.0,water,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,1.33,NaN,water,NaN,...,BIOSAMPLE:SAMEA1706311,449393,freshwater metagenome,Generic.1.0,water,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,2.77,NaN,water,NaN,...,BIOSAMPLE:SAMEA1706320,449393,freshwater metagenome,Generic.1.0,water,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,2.52,NaN,water,NaN,...,BIOSAMPLE:SAMEA1706300,449393,freshwater metagenome,Generic.1.0,water,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,3.4,NaN,water,NaN,...,BIOSAMPLE:SAMEA1706316,449393,freshwater metagenome,Generic.1.0,water,NaN,NaN,NaN,NaN,NaN


In [20]:
result.to_csv('../output/waterSampleWithEnitityPhrases.tsv', sep='\t', index=False)